In [1]:
import tensorflow as tf
from data.dataset import _get_training_data, _get_test_data
from DAE import DAE
import numpy as np



C:\Users\nambi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.app.flags.DEFINE_string('tf_records_train_path', 
                           'train/',
                           'Path of the training data.')

tf.app.flags.DEFINE_string('tf_records_test_path', 
                           'test/',
                           'Path of the test data.')

tf.app.flags.DEFINE_integer('num_epoch', 1000,
                            'Number of training epochs.')

tf.app.flags.DEFINE_integer('batch_size', 16,
                            'Size of the training batch.')

tf.app.flags.DEFINE_float('learning_rate',0.0005,
                          'Learning_Rate')

tf.app.flags.DEFINE_boolean('l2_reg', False,
                            'L2 regularization.'
                            )
tf.app.flags.DEFINE_float('lambda_',0.01,
                          'Wight decay factor.')

tf.app.flags.DEFINE_integer('num_v', 10000,
                            'Number of visible neurons (Number of movies the users rated.)')

tf.app.flags.DEFINE_integer('num_h', 128,
                            'Number of hidden neurons.)')

tf.app.flags.DEFINE_integer('num_samples', 10000,
                            'Number of training samples (Number of users, who gave a rating).')

FLAGS = tf.app.flags.FLAGS




In [3]:

def main(_):
    '''Building the graph, opening of a session and starting the training od the neural network.'''
    
    num_batches=int(FLAGS.num_samples/FLAGS.batch_size)

    with tf.Graph().as_default():

        train_data, train_data_infer=_get_training_data(FLAGS)
        test_data=_get_test_data(FLAGS)
        
        iter_train = train_data.make_initializable_iterator()
        iter_train_infer=train_data_infer.make_initializable_iterator()
        iter_test=test_data.make_initializable_iterator()
        
        x_train= iter_train.get_next()
        x_train_infer=iter_train_infer.get_next()
        x_test=iter_test.get_next()

        model=DAE(FLAGS)

        train_op, train_loss_op=model._optimizer(x_train)
        pred_op, test_loss_op=model._validation_loss(x_train_infer, x_test)
       
        with tf.Session() as sess:
            
            sess.run(tf.global_variables_initializer())
            train_loss=0
            test_loss=0

            for epoch in range(FLAGS.num_epoch):
                
                sess.run(iter_train.initializer)
                
                for batch_nr in range(num_batches):
#                     print (train_op)
#                     print (train_loss_op)
                    _, loss_=sess.run((train_op, train_loss_op))
                    train_loss+=loss_
              
                sess.run(iter_train_infer.initializer)
                sess.run(iter_test.initializer)

                for i in range(FLAGS.num_samples):
                    pred, loss_=sess.run((pred_op, test_loss_op))
                    test_loss+=loss_
                    
                print('epoch_nr: %i, train_loss: %.3f, test_loss: %.3f'%(epoch,(train_loss/num_batches),(test_loss/FLAGS.num_samples)))
                train_loss=0
                test_loss=0

if __name__ == "__main__":
    
    tf.app.run()

ValueError: Dimensions must be equal, but are 9999 and 10000 for 'inference/MatMul' (op: 'MatMul') with input shapes: [?,9999], [10000,128].